# 1. Setup

IMPORTANT!! You must be connected to a GPU or TPU in order to be able to load the Huggingface model.

# 2. Dependencies & pips

In [ ]:
#module4 part 1

!pip install llama-cpp-python==0.1.78

In [ ]:
# module 4 part 2
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

In [ ]:
!pip install streamlit

In [ ]:
! wget -w -O - ipv4.icanhazip.com

wget: --wait: Invalid time period ‘-O’


In [ ]:
!pip install pdf2image

In [ ]:
!pip install pytesseract

In [ ]:
!apt-get install tesseract-ocr

In [ ]:
!apt-get install poppler-utils


In [ ]:
!pip install numpy==1.23.5 pandas==1.5.3

# 3. Continue run here after session restart

In [ ]:
# module1
!pip install pymupdf #fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 88.0 MB/s eta 0:00:00


In [ ]:
#module2
!pip install nltk

# 4. Modules

In [ ]:
%%writefile load_hf.py
from huggingface_hub import login # notebook_login

def load_huggingface_accesstoken(token):
  """
  function loads huggingface access token (so you can get access to models such as Llama2, llama3)

  """
  try:
    login(token=token)
    print("Successfully logged in to Hugging Face")

  except Exception as e:
    print(f"Failed to log in to Hugging Face: {e}")

Writing load_hf.py


In [ ]:
%%writefile module1.py
import streamlit as st
import fitz
import pdf2image
import pytesseract
from PIL import Image


# Option 1. Read .txt file
def read_txt(txt_file):
  try:
    with open(txt_file, "r", encoding="utf-8") as f:
      readable_file = f.read()
  except Exception as e: # try another encoding (latin-1)
    st.error(f"Error handling TXT file with UTF-8 encoding: {e}")
    try:
      with open(txt_file, "r", encoding="latin-1") as f:
        readable_file= f.read()
    except Exception as e:
      st.error(f"Error reading TXT file with Latin-1 encoding: {e}")
      return None

  return readable_file


# Option 2. read .pdf file
def read_pdf(pdf_file):
    try:
        doc = fitz.open(stream= pdf_file.read(), filetype = "pdf") # pen .pdf file
        text = ""
        # iterate through each page of the .pdf
        for page in doc:
            text += page.get_text()  # extract text from the current page
        doc.close() # close doc
        return text

    except Exception as e:
        print(f"ERROR while reading PDF with fitz: {e}")
        return None


# ocr reader
def ocr_for_pdf(file, n_max_tokens =1000):
  st.info("OCR loading...")
  file.seek(0) # reset file pointer
  images = pdf2image.convert_from_bytes(file.read())
  complete_text = []
  #counter_pages=0
  for page_number, image in enumerate(images):
    #counter_pages = counter_pages +1
    #st.write("Page number:", counter_pages)
    #st.image(page, use_column_width = True) # display image
    text= pytesseract.image_to_string(image) # read text using pytesseract


    ## adustment to avoid loading and reading the whole paper: just get the first n tokens
    #complete_text.append(text) # this is to get the whole text
    tokens = text.split()
    num_tokens = len(tokens)


    if total_tokens + num_tokens < n_max_tokens:
      complete_text.append(text)
      total_tokens += num_tokens
    else:
      # calculate remaining_tokens
      remaining_tokens = n_max_tokens -total_tokens
      complete_text.append(" ").join(tokens[:remaining_tokens])
      break
    #num_pages = len(images)

  return " ".join(complete_text)


# Upload a file
def load_data():
    uploaded_data = st.file_uploader(":grey[Select your files]", type= ["pdf", "txt"], accept_multiple_files=True)
    content_dic = {}
    st.warning("Press the button once all the files are loaded")
    if st.button("Process Files"): #uploaded_data is not None:
      for file in uploaded_data:

        if file.name.endswith(".txt"):
            content = read_txt(txt_file = file)
            st.write(content)
            content_dic[file.name] = content

        elif file.name.endswith(".pdf"):
          try:
            # Read and print the text from the PDF
            content = read_pdf(pdf_file = file)

            if content is None:
                st.info("OCR is required. The process may take longer")
                final_text = ocr_for_pdf(file, n_max_tokens=1000)
                content = "\n".join(final_text) # output of ocr function is list, transform to string

            if content is not None:
                #st.write(content)
                content_dic[file.name] = content
            else:
              st.error(f"Failed to extract text from PDF: {file.name}")

          except Exception as e:
            st.error(f"An error occured while reading the pdf: {e}")

    #st.write("module 1",content_dic.keys(), content_dic.values())
    return content_dic

Overwriting module1.py


In [ ]:
%%writefile module2.py
import streamlit as st
import nltk
nltk.download("punkt")
#####


def shorten_content(content_dic: dict, n_max_tokens: int) -> dict:
  small_content_dic = {}
  for key, value in content_dic.items():
    tokens = nltk.word_tokenize(value)
    #num_tokens = len(tokens)
    small_content_dic[key] =  " ".join(tokens[:n_max_tokens]) # first n_max_tokens tokens

  return small_content_dic


def divide_content(content_dic: dict) -> dict:
    """Divide content into parts based on abstract and keyword sections."""
    content_dic_parts = {}

    abstract_list = ["abstract"]
    keywords_list = ["keywords"]

    for key, value in content_dic.items():
        tokens = nltk.word_tokenize(value)
        abstract_index = next((i for i, token in enumerate(tokens) if any(w in token.lower() for w in abstract_list)), None)
        keyword_index = next((i for i, token in enumerate(tokens) if any(w in token.lower() for w in keywords_list)), None)

        # debugging output
        print(f"Processing file: {key}")
        print(f"Total tokens: {len(tokens)}")
        print(f"Tokens preview: {tokens[:50]}...")  # print first 50 tokens
        print(f"Abstract index: {abstract_index}, Keyword index: {keyword_index}")

        # Initialize parts
        part1 = part2 = part3 = ""

        if abstract_index is not None and keyword_index is not None:
            part1 = " ".join(tokens[:abstract_index])
            part2 = " ".join(tokens[abstract_index:keyword_index])
            part3 = " ".join(tokens[keyword_index:])
        elif abstract_index is not None:
            part1 = " ".join(tokens[:abstract_index])
            part2 = " ".join(tokens[abstract_index:])
        elif keyword_index is not None:
            part1 = " ".join(tokens[:keyword_index])
            part3 = " ".join(tokens[keyword_index:])
        else:
            part1 = value  # all content goes directly to part1 if neither index is found

        # avoid duplication and ensure correct assignment
        if not part2 and not part3:
            print(f"Warning: No abstract or keywords found for file {key}")

        content_dic_parts[key] = [part1, part2, part3]

        # debugging output
        print(f"Parts for file {key}: Part 1 length = {len(part1)}, Part 2 length = {len(part2)}, Part 3 length = {len(part3)}")

    return content_dic_parts




def preprocess_data(content_dic):
  if content_dic == None:
    st.error("No files were uploaded")
    return None
  else:
    #st.write("Files were uploaded")
    smaller_content_dic = shorten_content(content_dic, n_max_tokens = 1000)
    content_dic_parts = divide_content(content_dic = smaller_content_dic)
  return content_dic_parts


Writing module2.py


In [ ]:
%%writefile module3.py

import pandas as pd
def create_csv_file():
  database = pd.DataFrame(columns=["file_id", "title", "author_name", "abstract", "keywords",
                          "journal", "volume", "proceedings",  "year", "month", "doi", "url", "pages", "field", "topic"]) # TODO: You can add as many columns as you want,but REMEMBER to add the prompt in the module 4
  return database


def fillin_database(database, content_dic):
  database["file_id"] = content_dic.keys()
  database["abstract"] = ""
  for key, value in content_dic.items():
    if len(value) == 3 and all(v is not None for v in value[:3]):
      database.loc[database["file_id"] == key, "abstract"] = value[1]
    # add abstract directly to content_dic for use in the next module!!
    content_dic[key].append(value[1])

  return database, content_dic  # return updated content_dic



Writing module3.py


In [ ]:
%%writefile module4.py

import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import torch

def load_model():
    model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
    model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"
    return model_name_or_path, model_basename

def model_gpu(model_path):
    lcpp_llm = Llama(
        model_path=model_path,
        n_threads=2,
        n_batch=128,
        n_gpu_layers=20,
        n_ctx=2048,
    )
    return lcpp_llm

prompt_dic = {
    "abstract": "If one, extract the abstract of this text and return it. Otherwise return NONE",
    "author_name": "If one, extract the author/s names of this text. Otherwise return NONE",
    "doi": "If one, extract the DOI of this text. Otherwise return NONE",
    "title": "If one, extract the title of this text. Otherwise return NONE",
    "keywords": "If one, extract the keywords from the Keyword section of this text. Otherwise return NONE",
    "pages": "If one, extract the pages where this article was published. Otherwise return NONE",
    "year": "If one, extract ONLY the year of publication of this text. Otherwise return NONE",
    "month": "If one, extract ONLY the month of publication of this text. Otherwise return NONE",
    "url": "If one, extract ONLY the URL of publication of this text. Otherwise return NONE",
    "journal": "If one, extract ONLY the journal name of publication of this text",
    "volume": "If one, extract ONLY the Volume number or issue of publication of this text. Otherwise return NONE",
    "proceedings": "If one, extract ONLY the Proceedings where this text was published. If it is other than Proceedings (example:journal), return NONE"
}

def llm_content_extractor(gpu_for_model, database, content_dic, prompt_dic):
    for pdf_name, parts in content_dic.items():
        for key, prompt in prompt_dic.items():
            if key == "abstract":
                abstract_entry = database.loc[database["file_id"] == pdf_name, key]
                if not abstract_entry.empty and abstract_entry.iloc[0] != "":
                    continue

            full_prompt = ""
            if key == "abstract":
                full_prompt = f"{prompt_dic[key]} [TEXT]{parts[1] if parts[1] else parts[0]}[TEXT]"
            elif key == "keywords":
                full_prompt = f"{prompt_dic[key]} [TEXT]{parts[2] if parts[2] else (parts[1] if parts[1] else parts[0])}[TEXT]"
            else:
                full_prompt = f"{prompt_dic[key]} [TEXT]{parts[0] if parts[0] else parts[1]}[TEXT]"

            prompt_template = f'''SYSTEM: You are a Bibliography assistant. You give ONLY the answer as found. You NEVER add any extra words.
                                  USER: {full_prompt}
                                  ASSISTANT:
            '''
            try:
                response = gpu_for_model(
                    prompt=prompt_template,
                    max_tokens=256,
                    temperature=0.5,
                    top_p=0.95,
                    repeat_penalty=1.2,
                    top_k=50,
                    stop=['USER:', 'SYSTEM:', 'ASSISTANT:'],
                    echo=True
                )

                assistant_response = response["choices"][0]["text"].strip()
                assistant_index = assistant_response.find("ASSISTANT:")
                if assistant_index != -1:
                    assistant_response = assistant_response[assistant_index + len("ASSISTANT:"):].strip()

                database.loc[database["file_id"] == pdf_name, key] = assistant_response

                torch.cuda.empty_cache()

            except Exception as e:
                print(f"Error processing {key} for {pdf_name}: {e}")
                database.loc[database["file_id"] == pdf_name, key] = "Error"

    return database


def main_llmExtractor(database, content_dic):
    model_name_or_path, model_basename = load_model()
    try:
        model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
        print(f"Model downloaded to: {model_path}")
    except Exception as e:
        print(f"Failed to download model: {e}")
        return database

    try:
        gpu_for_model = model_gpu(model_path=model_path)
    except Exception as e:
        print(f"Failed to load model to GPU: {e}")
        return database

    print("module 4 main_llmExtractor")

    # debug the content_dic and database before processing
    print("Initial database:")
    print(database.head())
    print("Initial content_dic:")
    print(content_dic)

    # process data
    database = llm_content_extractor(gpu_for_model, database, content_dic, prompt_dic)

    # debug the processed database
    print("Processed database:")
    print(database.head())

    return database


Writing module4.py


# 5. app.py

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd

from load_hf import load_huggingface_accesstoken
from module1 import load_data
from module2 import preprocess_data
from module3 import create_csv_file, fillin_database
from module4 import main_llmExtractor


st.set_page_config(page_title='Citeasyon', page_icon=":tada:", layout="wide")
# HEADER
with st.container():
    st.title(":red[_CITEASYON step 1_] :sparkles:")
    #st.subheader(""":grey[Welcome to _your PDF to text converter_ for FREE]""", divider="orange")



def run_citeasyon():
  ## 0. Set up hugging face access token
  token = st.text_input("Enter your Hugging Face access Token", type= "password")
  if token:
    load_huggingface_accesstoken(token)
  else:
    st.warning("Access token for Hugging Face required")
  #load_huggingface_accesstoken()
  #st.write("0. Set up hf")

  ## 1. Load data
  #st.write("1. Load data")
  content_dic = load_data()


  ## 2. Get 1000 first tokens, and get parts (up to asbtract, from abstract onwards)
  #st.write("2. Get 1000 first tokens")
  content_dic_parts = preprocess_data(content_dic= content_dic)
  #print("Content preprocessed -- OCR -- TXT --- ")

  # start debugging module 2
  #st.write(content_dic_parts.keys(), content_dic_parts.values())
  #st.write("module 2: getting 3 parts:")
  #parts_list = [(key, value[0], value[1], value[2]) for key, value in content_dic_parts.items()]
  #st.write(parts_list)
  # end debugging module 2


  # 3. Create .csv file to store information & fill in with file names, and columns: title, authors, journal, year, month, abstract, DOI, URL,...
  database = create_csv_file()
  #st.write("Database created")

  data_csv, content_dic_parts = fillin_database(database= database, content_dic = content_dic_parts)
  st.write("Files to process:", data_csv)

  # 4. LLM extract information & paste into .csv file
  structured_content = main_llmExtractor(database=data_csv, content_dic=content_dic_parts)
  st.write("Structured_content", structured_content)

  st.download_button("Download citeasyon databse", data=structured_content.to_csv(), file_name= "citeasyon_database.csv")
  st.write("Now, take your citeasyon_database.csv and go to the NEXT STEP (Citeasyon step 2) to get your citatiosn into MLA or APA format")

  return structured_content


run_citeasyon()


Overwriting app.py


# 6. Run local tunnel

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501
